[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/VectorInstitute/rag-bootcamp/blob/refactor/uv-migration/implementations/pubmed_qa/pubmed_qa_llamaindex.ipynb)

# PubMed QA using LlamaIndex

## Introduction
This notebook presents a RAG workflow for the [PubMed QA](https://pubmedqa.github.io/) task using [LlamaIndex](https://www.llamaindex.ai/). The code is written in a configurable fashion, giving you the flexibility to edit the RAG configuration and observe the change in output/responses.

It covers a step-by-step procedure for building the RAG workflow (Stages 1-4) and later runs the pipeline on a sample from the dataset. The notebook also covers the sparse, dense, hybrid retrieval strategies along with the re-ranker. We have alse added an optional component for RAG evaluation using the [Ragas](https://docs.ragas.io/en/stable/) library.

### 📝 Requirements

To run this notebook, you will need:

- **OpenAI API key**:  
    - Sign up at [OpenAI](https://platform.openai.com/) and create an API key

- **Weaviate vector database cluster**:  
    - Create a free Weaviate Cloud Service (WCS) cluster at [Weaviate Cloud Console](https://console.weaviate.io/)
    - After creation, copy the cluster URL (e.g., `https://your-cluster-id.weaviate.network`) and API key (Admin)


## STAGE 0 - Set up the RAG workflow environment

#### Install libraries (Only in Google Colab)

In [ ]:
import os

if "COLAB_RELEASE_TAG" in os.environ:
    # This is a Google Colab environment
    # Install required dependencies
    !pip3 install faiss-cpu langchain llama-index llama-index-core llama-index-embeddings-huggingface llama-index-llms-openai-like llama-index-vector-stores-faiss # aieng-rag-utils

#### Import libraries, custom classes and functions

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import os
import random

from pprint import pprint

from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter

from aieng.rag.utils.pubmed import (
    PubMedQATaskDataset,
    RAGLLM,
    DocumentReader,
    RAGEmbedding,
    RAGQueryEngine,
    RagasEval,
    RAGIndex,
    extract_yes_no,
    validate_rag_cfg,
)

#### Load OpenAI and Weaviate env variables

In [ ]:
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY")
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY", "YOUR_WEAVIATE_API_KEY")
WEAVIATE_CLUSTER_URL = os.getenv(
    "WEAVIATE_CLUSTER_URL", "https://YOUR_WEAVIATE_CLUSTER_URL"
)

# Set it as an environment variables if not already set
os.environ["OPENAI_BASE_URL"] = OPENAI_BASE_URL
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["WEAVIATE_API_KEY"] = WEAVIATE_API_KEY
os.environ["WEAVIATE_CLUSTER_URL"] = WEAVIATE_CLUSTER_URL

#### Set RAG configuration

In [ ]:
rag_cfg = {
    # Node parser config
    "chunk_size": 256,
    "chunk_overlap": 0,
    # Embedding model config
    "embed_model_type": "hf",
    "embed_model_name": "BAAI/bge-base-en-v1.5",
    # LLM config
    "llm_type": "openai",
    "llm_name": "gpt-4o",
    "max_new_tokens": 256,
    "temperature": 0.0,
    "top_p": 1.0,
    "top_k": 50,
    "do_sample": False,
    # Vector DB config
    "vector_db_type": "weaviate",  # "weaviate"
    "vector_db_name": "Pubmed_QA",
    # MODIFY THIS
    "weaviate_url": WEAVIATE_CLUSTER_URL,
    # Retriever and query config
    "retriever_type": "vector_index",  # "vector_index"
    "retriever_similarity_top_k": 5,
    "query_mode": "hybrid",  # "default", "hybrid"
    "hybrid_search_alpha": 0.0,  # float from 0.0 (sparse search - bm25) to 1.0 (vector search)
    "response_mode": "compact",
    "use_reranker": False,
    "rerank_top_k": 3,
    # Evaluation config
    "eval_llm_type": "openai",
    "eval_llm_name": "gpt-4.1",
}

#### Preliminary config checks

In [6]:
validate_rag_cfg(rag_cfg)
pprint(rag_cfg)

{'chunk_overlap': 0,
 'chunk_size': 256,
 'do_sample': False,
 'embed_model_name': 'BAAI/bge-base-en-v1.5',
 'embed_model_type': 'hf',
 'eval_llm_name': 'gpt-4.1',
 'eval_llm_type': 'openai',
 'hybrid_search_alpha': 0.0,
 'llm_name': 'gpt-4o',
 'llm_type': 'openai',
 'max_new_tokens': 256,
 'query_mode': 'hybrid',
 'rerank_top_k': 3,
 'response_mode': 'compact',
 'retriever_similarity_top_k': 5,
 'retriever_type': 'vector_index',
 'temperature': 0.0,
 'top_k': 50,
 'top_p': 1.0,
 'use_reranker': False,
 'vector_db_name': 'Pubmed_QA',
 'vector_db_type': 'weaviate',
 'weaviate_url': 'https://4rpjojus1cb87lxh3gi7w.c0.us-west3.gcp.weaviate.cloud'}


## STAGE 1 - Load dataset and documents

#### 1. Load PubMed QA dataset
PubMedQA ([github](https://github.com/pubmedqa/pubmedqa)) is a biomedical question answering dataset. Each instance consists of a question, a context (extracted from PubMed abstracts), a long answer and a yes/no/maybe answer. We make use of the test split of [this](https://huggingface.co/datasets/bigbio/pubmed_qa) huggingface dataset for this notebook.

**The context for each instance is stored as a text file** (referred to as documents), to align the task as a standard RAG use-case.

In [7]:
print("Loading PubMed QA data ...")
pubmed_data = PubMedQATaskDataset("bigbio/pubmed_qa")
print(f"Loaded data size: {len(pubmed_data)}")
pubmed_data.mock_knowledge_base(output_dir="./data", one_file_per_sample=True)

Loading PubMed QA data ...


Preparing data: 100%|██████████| 500/500 [00:00<00:00, 3734.05it/s]

Loaded data size: 500


#### 2. Load documents
All metadata is excluded by default. Set the *exclude_llm_metadata_keys* and *exclude_embed_metadata_keys* flags to *false* for including it. Please refer to [this](https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents.html) and the *DocumentReader* class from *rag_utils.py* for further details.

In [8]:
print("Loading documents ...")
reader = DocumentReader(input_dir="./data/pubmed_doc")
docs = reader.load_data()
print(f"No. of documents loaded: {len(docs)}")

Loading documents ...
No. of documents loaded: 500


## STAGE 2 - Load node parser, embedding, LLM and set service context

#### 1. Load node parser to split documents into smaller chunks

In [9]:
print("Loading node parser ...")
node_parser = SentenceSplitter(
    chunk_size=rag_cfg["chunk_size"], chunk_overlap=rag_cfg["chunk_overlap"]
)
nodes = node_parser.get_nodes_from_documents(docs)

Loading node parser ...


#### 2. Load embedding model
LlamaIndex supports embedding models from OpenAI, Cohere, HuggingFace, etc. Please refer to [this](https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#custom-embedding-model) for building a custom embedding model.

In [10]:
embed_model = RAGEmbedding(
    model_type=rag_cfg["embed_model_type"], model_name=rag_cfg["embed_model_name"]
).load_model()

Loading hf embedding model ...


#### 3. Load LLM for generation
LlamaIndex supports LLMs from OpenAI, Cohere, HuggingFace, AI21, etc. Please refer to [this](https://docs.llamaindex.ai/en/stable/module_guides/models/llms/usage_custom.html#example-using-a-custom-llm-model-advanced) for loading a custom LLM model for generation.

In [11]:
llm = RAGLLM(
    llm_type=rag_cfg["llm_type"],
    llm_name=rag_cfg["llm_name"],
    api_base=OPENAI_BASE_URL,
    api_key=OPENAI_API_KEY,
).load_model(**rag_cfg)

Configuring openai LLM model ...


#### 4. Use ```Settings``` to set the node parser, embedding model, LLM, etc.

In [12]:
Settings.text_splitter = node_parser
Settings.llm = llm
Settings.embed_model = embed_model

## STAGE 3 - Create index using the appropriate vector store
All vector stores supported by LlamaIndex along with their available features are listed [here](https://docs.llamaindex.ai/en/stable/module_guides/storing/vector_stores.html).

If you are using LangChain, the supported vector stores can be found [here](https://python.langchain.com/docs/modules/data_connection/vectorstores/).

In [13]:
index = RAGIndex(
    db_type=rag_cfg["vector_db_type"],
    db_name=rag_cfg["vector_db_name"],
).create_index(docs, weaviate_url=rag_cfg["weaviate_url"])

/Users/aishwaryabannimatti/Documents/work/rag-bootcamp/.venv/lib/python3.10/site-packages/weaviate/warnings.py:133: DeprecationWarning: Dep005: You are using weaviate-client version 4.9.3. The latest version is 4.14.4.
            Consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


Loading index from ./.weaviate_index_store/ ...


## STAGE 4 - Build query engine

Now build a query engine using *retriever* and *response_synthesizer*. LlamaIndex also supports different types of [retrievers](https://docs.llamaindex.ai/en/stable/api_reference/query/retrievers.html) and [response modes](https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/root.html#configuring-the-response-mode) for various use-cases.

[Weaviate hybrid search](https://weaviate.io/blog/hybrid-search-explained) explains how dense and sparse search is combined.

In [14]:
def set_query_engine_args(rag_cfg, docs):
    query_engine_args = {
        "similarity_top_k": rag_cfg["retriever_similarity_top_k"],
        "response_mode": rag_cfg["response_mode"],
        "use_reranker": False,
    }

    if (rag_cfg["retriever_type"] == "vector_index") and (
        rag_cfg["vector_db_type"] == "weaviate"
    ):
        query_engine_args.update(
            {
                "query_mode": rag_cfg["query_mode"],
                "hybrid_search_alpha": rag_cfg["hybrid_search_alpha"],
            }
        )
    elif rag_cfg["retriever_type"] == "bm25":
        nodes = Settings.text_splitter.get_nodes_from_documents(docs)
        tokenizer = Settings.embed_model._tokenizer
        query_engine_args.update({"nodes": nodes, "tokenizer": tokenizer})

    if rag_cfg["use_reranker"]:
        query_engine_args.update(
            {"use_reranker": True, "rerank_top_k": rag_cfg["rerank_top_k"]}
        )

    return query_engine_args

In [15]:
query_engine_args = set_query_engine_args(rag_cfg, docs)
pprint(query_engine_args)

{'hybrid_search_alpha': 0.0,
 'query_mode': 'hybrid',
 'response_mode': 'compact',
 'similarity_top_k': 5,
 'use_reranker': False}


In [16]:
query_engine = RAGQueryEngine(
    retriever_type=rag_cfg["retriever_type"],
    vector_index=index,
).create(**query_engine_args)

## STAGE 5 - Finally query the model !
**Note:** We are using keyword based search or sparse search since *hybrid_search_alpha* is set to 0.0 by default.

#### [TODO] Change seed to experiment with a different sample

In [17]:
random.seed(237)

In [18]:
sample_idx = random.randint(0, len(pubmed_data) - 1)
sample_elm = pubmed_data[sample_idx]
pprint(sample_elm)

{'answer': ['no'],
 'context': 'Human immunodeficiency virus (HIV)-infected patients have '
            'generally been excluded from transplantation. Recent advances in '
            'the management and prognosis of these patients suggest that this '
            'policy should be reevaluated. To explore the current views of '
            'U.S. transplant centers toward transplanting asymptomatic '
            'HIV-infected patients with end-stage renal disease, a written '
            'survey was mailed to the directors of transplantation at all 248 '
            'renal transplant centers in the United States. All 148 responding '
            'centers said they require HIV testing of prospective kidney '
            'recipients, and 84% of these centers would not transplant an '
            'individual who refuses HIV testing. The vast majority of '
            'responding centers would not transplant a kidney from a cadaveric '
            '(88%) or a living donor (91%) into an asymp

In [19]:
query = sample_elm["question"]

response = query_engine.query(query)

delim = "".join(["-"] * 25)
print(f"QUERY: {query}\n")
print(f"RESPONSE:\n{delim}\n{response.response}\n{delim}\n")
print(f"YES/NO: {extract_yes_no(response.response)}\n")
print(f"GT ANSWER: {sample_elm['answer'][0]}\n")
print(f"GT LONG ANSWER:\n{delim}\n{sample_elm['long_answer']}\n{delim}")

QUERY: Should all human immunodeficiency virus-infected patients with end-stage renal disease be excluded from transplantation?

RESPONSE:
-------------------------
The context information suggests that the exclusion of all HIV-infected patients with end-stage renal disease from transplantation may not be justified. Recent advances in the management and prognosis of HIV-infected patients indicate that this policy should be reevaluated. While the majority of U.S. transplant centers currently do not perform transplants on asymptomatic HIV-infected patients, there is a recognition that these patients could potentially be good candidates for transplantation. The main concerns are related to the potential harm to the individual and the perceived waste of organs. However, with improved management of HIV, these concerns may be mitigated, suggesting that a blanket exclusion may not be appropriate. Therefore, the decision should be individualized, taking into account the specific health status 

#### [OPTIONAL] [Ragas](https://docs.ragas.io/en/latest/) evaluation
Following are the commonly used metrics for evaluating a RAG workflow:
* [Faithfulness](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/faithfulness/): Measures the factual correctness of the generated answer based on the retrived context. Value lies between 0 and 1. **Evaluated using a LLM.**
* [Answer Relevance](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/answer_relevance/): Measures how relevant the answer is to the given query. Value lies between 0 and 1. **Evaluated using a LLM.**
* [Context Precision](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/context_precision/): Precision of the retriever as measured using the retrieved and the ground truth context. Value lies between 0 and 1. LLM can be used for evaluation.
* [Context Recall](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/context_recall/): Recall of the retriever as measured using the retrieved and the ground truth context. Value lies between 0 and 1. LLM can be used for evaluation.

In [20]:
retrieved_nodes = query_engine.retriever.retrieve(query)

eval_data = [
    dict(
        {
            "user_input": query,
            "response": response.response,
            "retrieved_contexts": [node.text for node in retrieved_nodes],
            "reference": sample_elm["long_answer"],
            "reference_contexts": [sample_elm["context"]],
        }
    )
]
pprint(eval_data)

[{'reference': 'The great majority of U.S. renal transplant centers will not '
               'transplant kidneys to HIV-infected patients with end-stage '
               'renal disease, even if their infection is asymptomatic. '
               'However, advances in the management of HIV infection and a '
               'review of relevant ethical issues suggest that this approach '
               'should be reconsidered.',
  'reference_contexts': ['Human immunodeficiency virus (HIV)-infected patients '
                         'have generally been excluded from transplantation. '
                         'Recent advances in the management and prognosis of '
                         'these patients suggest that this policy should be '
                         'reevaluated. To explore the current views of U.S. '
                         'transplant centers toward transplanting asymptomatic '
                         'HIV-infected patients with end-stage renal disease, '
                

In [21]:
eval_obj = RagasEval(
    metrics=["faithfulness", "relevancy", "recall", "precision"],
    eval_llm_type=rag_cfg["eval_llm_type"],
    eval_llm_name=rag_cfg["eval_llm_name"],
    embed_model_name=rag_cfg["embed_model_name"],
    max_tokens=1024,
)

In [22]:
eval_result = eval_obj.evaluate(eval_data)
pprint(eval_result)

/Users/aishwaryabannimatti/Documents/work/rag-bootcamp/.venv/lib/python3.10/site-packages/ragas/_analytics.py:77: ResourceWarning: unclosed file <_io.TextIOWrapper name='/Users/aishwaryabannimatti/Library/Application Support/ragas/uuid.json' mode='r' encoding='UTF-8'>
  user_id = json.load(open(uuid_filepath))["userid"]


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Exception raised in Job[0]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)


{'faithfulness': nan, 'answer_relevancy': 0.9855, 'non_llm_context_recall': 1.0000, 'non_llm_context_precision_with_reference': 1.0000}


### 5.1 - Dense Search
Set *hybrid_search_alpha* to 1.0 for dense vector search.

In [23]:
rag_cfg["hybrid_search_alpha"] = 1.0

In [24]:
# Recreate query engine
query_engine_args = set_query_engine_args(rag_cfg, docs)
pprint(query_engine_args)
query_engine = RAGQueryEngine(
    retriever_type=rag_cfg["retriever_type"], vector_index=index
).create(**query_engine_args)

# Get response
response = query_engine.query(query)

# Print response
print(f"\n\nQUERY: {query}\n")
print(f"RESPONSE:\n{delim}\n{response.response}\n{delim}\n")
print(f"YES/NO: {extract_yes_no(response.response)}\n")
print(f"GT ANSWER: {sample_elm['answer'][0]}\n")
print(f"GT LONG ANSWER:\n{delim}\n{sample_elm['long_answer']}\n{delim}")

{'hybrid_search_alpha': 1.0,
 'query_mode': 'hybrid',
 'response_mode': 'compact',
 'similarity_top_k': 5,
 'use_reranker': False}


QUERY: Should all human immunodeficiency virus-infected patients with end-stage renal disease be excluded from transplantation?

RESPONSE:
-------------------------
The context information suggests that the exclusion of all HIV-infected patients with end-stage renal disease from transplantation should be reevaluated. Recent advances in the management and prognosis of HIV-infected patients indicate that they may be viable candidates for transplantation. Although the majority of U.S. transplant centers currently do not perform transplants on asymptomatic HIV-infected patients, there is a recognition that this policy might need to change. The fear that transplantation could be harmful or a waste of organs is prevalent, but it is not universally supported by evidence, especially given the improvements in HIV treatment and management.

Therefore, the context i

#### [OPTIONAL] Ragas evaluation

In [25]:
retrieved_nodes = query_engine.retriever.retrieve(query)

eval_data = [
    dict(
        {
            "user_input": query,
            "response": response.response,
            "retrieved_contexts": [node.text for node in retrieved_nodes],
            "reference": sample_elm["long_answer"],
            "reference_contexts": [sample_elm["context"]],
        }
    )
]

eval_result = eval_obj.evaluate(eval_data)
pprint(eval_result)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'faithfulness': 0.6364, 'answer_relevancy': 0.9855, 'non_llm_context_recall': 1.0000, 'non_llm_context_precision_with_reference': 1.0000}


### 5.2 - Hybrid Search
Set *hybrid_search_alpha* to 0.5 for hybrid search with equal weightage for dense and sparse (keyword-based) search.

In [26]:
rag_cfg["hybrid_search_alpha"] = 0.5

In [27]:
# Recreate query engine
query_engine_args = set_query_engine_args(rag_cfg, docs)
pprint(query_engine_args)
query_engine = RAGQueryEngine(
    retriever_type=rag_cfg["retriever_type"], vector_index=index
).create(**query_engine_args)

# Get response
response = query_engine.query(query)

# Print response
print(f"\n\nQUERY: {query}\n")
print(f"RESPONSE:\n{delim}\n{response.response}\n{delim}\n")
print(f"YES/NO: {extract_yes_no(response.response)}\n")
print(f"GT ANSWER: {sample_elm['answer'][0]}\n")
print(f"GT LONG ANSWER:\n{delim}\n{sample_elm['long_answer']}\n{delim}")

{'hybrid_search_alpha': 0.5,
 'query_mode': 'hybrid',
 'response_mode': 'compact',
 'similarity_top_k': 5,
 'use_reranker': False}


QUERY: Should all human immunodeficiency virus-infected patients with end-stage renal disease be excluded from transplantation?

RESPONSE:
-------------------------
The context information suggests that the exclusion of all HIV-infected patients with end-stage renal disease from transplantation may not be justified. Recent advances in the management and prognosis of HIV-infected patients indicate that this policy should be reevaluated. While the majority of U.S. transplant centers currently do not perform transplants on asymptomatic HIV-infected patients, there is a recognition that these patients could potentially be good candidates for transplantation. The main concerns are the potential harm to the individual and the perceived waste of organs. However, with improved HIV management, these concerns may be mitigated, suggesting that a blanket exclusion ma

#### [OPTIONAL] Ragas evaluation

In [28]:
retrieved_nodes = query_engine.retriever.retrieve(query)

eval_data = [
    dict(
        {
            "user_input": query,
            "response": response.response,
            "retrieved_contexts": [node.text for node in retrieved_nodes],
            "reference": sample_elm["long_answer"],
            "reference_contexts": [sample_elm["context"]],
        }
    )
]

eval_result = eval_obj.evaluate(eval_data)
pprint(eval_result)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'faithfulness': 0.7273, 'answer_relevancy': 0.9855, 'non_llm_context_recall': 1.0000, 'non_llm_context_precision_with_reference': 1.0000}


### 5.3 - Using Re-ranker
Set *use_reranker* to *True* to re-rank the context after retrieving it from the vector database.

In [29]:
rag_cfg["use_reranker"] = True
rag_cfg["hybrid_search_alpha"] = 1.0  # Using dense search

In [30]:
# Recreate query engine
query_engine_args = set_query_engine_args(rag_cfg, docs)
pprint(query_engine_args)
query_engine = RAGQueryEngine(
    retriever_type=rag_cfg["retriever_type"], vector_index=index
).create(**query_engine_args)

# Get response
response = query_engine.query(query)

# Print response
print(f"\n\nQUERY: {query}\n")
print(f"RESPONSE:\n{delim}\n{response.response}\n{delim}\n")
print(f"YES/NO: {extract_yes_no(response.response)}\n")
print(f"GT ANSWER: {sample_elm['answer'][0]}\n")
print(f"GT LONG ANSWER:\n{delim}\n{sample_elm['long_answer']}\n{delim}")

{'hybrid_search_alpha': 1.0,
 'query_mode': 'hybrid',
 'rerank_top_k': 3,
 'response_mode': 'compact',
 'similarity_top_k': 5,
 'use_reranker': True}


QUERY: Should all human immunodeficiency virus-infected patients with end-stage renal disease be excluded from transplantation?

RESPONSE:
-------------------------
The context information suggests that the policy of excluding HIV-infected patients from transplantation is being reevaluated due to recent advances in the management and prognosis of these patients. However, the majority of U.S. transplant centers still do not support transplanting kidneys into asymptomatic HIV-infected patients, citing concerns about potential harm to the individual and the belief that it might be a waste of organs. Despite these concerns, the fact that some centers are considering the possibility indicates that there is not a unanimous agreement on the exclusion policy.

Given this context, it seems that a blanket exclusion of all HIV-infected patients fr

#### [OPTIONAL] Ragas evaluation

In [31]:
retrieved_nodes = query_engine.retriever.retrieve(query)

eval_data = [
    dict(
        {
            "user_input": query,
            "response": response.response,
            "retrieved_contexts": [node.text for node in retrieved_nodes],
            "reference": sample_elm["long_answer"],
            "reference_contexts": [sample_elm["context"]],
        }
    )
]

eval_result = eval_obj.evaluate(eval_data)
pprint(eval_result)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'faithfulness': 0.6364, 'answer_relevancy': 0.9855, 'non_llm_context_recall': 1.0000, 'non_llm_context_precision_with_reference': 1.0000}
